In [1]:
#!/usr/bin/python
# -*- encoding: utf-8 -*-
'''
@File    :   单卡加载多卡训练的模型.ipynb
@Time    :   2024/05/19 10:25:31
@Author  :   Hengda.Gao
@Contact :   ghd@nudt.edu.com
@Desc    :   解决###    failed load 问题？？
                        Failed loading state dict.
                        Failed loading state dict.d
'''
import os
import torch

In [9]:
path = '/home/hengda/VideoMetamaterials/runs/debug-128/model/step_200000/checkpoint-ori.pt' # path to the multigpu-trained checkpoint file
if not os.path.isfile(path):
    raise FileNotFoundError(
        f'trainer checkpoint not found at {path}. Please check path or run load_model_step = None'
    )

# to avoid extra GPU memory usage in main process when using Accelerate
# 为了避免在使用Accelerate时主进程中出现额外的GPU内存使用，使用torch.load函数的map_location参数将模型加载到CPU上
with open(path, 'rb') as f:
    loaded_obj = torch.load(f, map_location='cpu')

In [6]:
def remove_module(state_dict):
    """
    Removes 'module.' from the keys of the state_dict.
    """
    from collections import OrderedDict
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        if k.startswith('module.'):
            name = k[7:]            # remove `module.`
            new_state_dict[name] = v
        else:
            print(f'key:{k})')
            new_state_dict[k] = v
    return new_state_dict

In [7]:
print(loaded_obj.keys())
print("*"*100)
for key in loaded_obj.keys():
    if key == 'steps':
        continue
    print(f'去掉module之前的{key}.keys:')
    print(loaded_obj[key].keys())
    if key in ['model', 'ema']:
        # 去除modules
        loaded_obj[key] = remove_module(loaded_obj[key])
        print(f'去掉module之后的{key}.keys:')
        print(loaded_obj[key].keys())

    print("*"*100)

dict_keys(['model', 'optimizer', 'steps', 'ema'])
****************************************************************************************************
去掉module之前的model.keys:
odict_keys(['module.betas', 'module.alphas_cumprod', 'module.alphas_cumprod_prev', 'module.sqrt_alphas_cumprod', 'module.sqrt_one_minus_alphas_cumprod', 'module.log_one_minus_alphas_cumprod', 'module.sqrt_recip_alphas_cumprod', 'module.sqrt_recipm1_alphas_cumprod', 'module.posterior_variance', 'module.posterior_log_variance_clipped', 'module.posterior_mean_coef1', 'module.posterior_mean_coef2', 'module.denoise_fn.null_text_token', 'module.denoise_fn.null_text_hidden', 'module.denoise_fn.time_rel_pos_bias.relative_attention_bias.weight', 'module.denoise_fn.init_conv.weight', 'module.denoise_fn.init_conv.bias', 'module.denoise_fn.init_temporal_attn.fn.fn.fn.rotary_emb.freqs', 'module.denoise_fn.init_temporal_attn.fn.fn.fn.to_qkv.weight', 'module.denoise_fn.init_temporal_attn.fn.fn.fn.to_q.weight', 'module.denoise_fn.

In [8]:
save_dir = '/home/hengda/VideoMetamaterials/runs/debug-128/model/step_200000/checkpoint.pt'
save_obj = loaded_obj
with open(save_dir, 'wb') as f:
    torch.save(save_obj, f)